In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [2]:
testdata = 'target area: x=20..30, y=-10..-5'
puzzledata = 'target area: x=88..125, y=-157..-103'

## part 1 ##

In [3]:
def parse_target(s):
    xlim, zlim = s[12:].split(',')
    xnums = xlim.split('=')[1]
    xmin, xmax = xnums.split('..')
    znums = zlim.split('=')[1]
    zmin, zmax = znums.split('..')
    return int(xmin), int(xmax), int(zmin), int(zmax)

In [4]:
parse_target(testdata)

(20, 30, -10, -5)

In [5]:
def xtraj(vx0, steps):
    x, vx = 0, vx0
    xpts = [x]
    for j in range(steps):
        if vx > 0:
            x += vx
            vx -= 1
        else:
            pass
        xpts.append(x)
    return xpts
    
def ztraj(vz0, steps):
    z, vz = 0, vz0
    zpts = [z]
    for j in range(steps):
        z += vz
        vz -= 1
        zpts.append(z)
    return zpts

def traj(vx0, vz0, steps):
    return list(zip(xtraj(vx0, steps), ztraj(vz0, steps)))

In [6]:
def find_valid_vx0(target_xmin, target_xmax):
    valid = []
    for vx0 in range(1, target_xmax+1):
        xpts = xtraj(vx0, target_xmin)
        if any(target_xmin <= x <= target_xmax for x in xpts):
            valid.append(vx0)
    return valid

In [7]:
bignum = 1000
def find_valid_vz0(target_zmin, target_zmax):
    valid = []
    for vz0 in range(1, bignum):
        zpts = ztraj(vz0, bignum)
        if any(target_zmin <= z <= target_zmax for z in zpts):
            valid.append(vz0)
    return valid

In [8]:
def step(x, z, vx, vz):
    if vx > 0:
        x += vx
        vx -= 1
    z += vz
    vz -= 1
    return x, z, vx, vz

In [9]:
def solve(data):
    target_xmin, target_xmax, target_zmin, target_zmax = parse_target(data)
    initconds = itertools.product(find_valid_vx0(target_xmin, target_xmax),
                                  find_valid_vz0(target_zmin, target_zmax))
    maxz = 0
    maxz_conds = None
    for vx0, vz0 in initconds:
        x, z = 0, 0
        vx, vz = vx0, vz0
        curr_max_z = 0
        hit_target = False
        while True:
            x, z, vx, vz = step(x, z, vx, vz)
            if z > curr_max_z:
                curr_max_z = z
            if x > target_xmax:
                break
            if z < target_zmin:
                break
            if (target_xmin <= x <= target_xmax) and (target_zmin <= z <= target_zmax):
                hit_target = True
        if hit_target:
            if curr_max_z > maxz:
                maxz = curr_max_z
                maxz_conds = vx0, vz0
    return maxz, maxz_conds

In [10]:
solve(testdata)

(45, (6, 9))

In [11]:
solve(puzzledata)

(12246, (13, 156))

## part 2 ##

In [12]:
bignum = 1000
def find_valid_vz0_2(target_zmin, target_zmax):
    # negative vz0 will work, now. Lower limit is target_zmin, since 1
    valid = []
    for vz0 in range(target_zmin, bignum):
        zpts = ztraj(vz0, bignum)
        if any(target_zmin <= z <= target_zmax for z in zpts):
            valid.append(vz0)
    return valid

In [13]:
def solve2(data):
    target_xmin, target_xmax, target_zmin, target_zmax = parse_target(data)
    initconds = itertools.product(find_valid_vx0(target_xmin, target_xmax),
                                  find_valid_vz0_2(target_zmin, target_zmax))
    valid = []
    for vx0, vz0 in initconds:
        x, z = 0, 0
        vx, vz = vx0, vz0
        hit_target = False
        while True:
            x, z, vx, vz = step(x, z, vx, vz)
            if x > target_xmax:
                break
            if z < target_zmin:
                break
            if (target_xmin <= x <= target_xmax) and (target_zmin <= z <= target_zmax):
                hit_target = True
                valid.append((vx0, vz0))
                break
    return len(valid)

In [14]:
solve2(testdata)

112

In [15]:
solve2(puzzledata)

3528